In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Define a function to load data from multiple files
def load_activity_data(activity, file_prefix, num_files):
    data_list = []
    for i in range(1, num_files + 1):
        file_path = f"Data/{activity}/{file_prefix} {i}cleaned.csv"
        data = np.genfromtxt(file_path, delimiter=',')
        data_list.append(data)
    return np.concatenate(data_list, axis=0)

# Load data for each activity
reading_data = load_activity_data("Reading", "r", 6)
speaking_data = load_activity_data("Speaking", "s", 4)
watching_data = load_activity_data("Watching", "w", 6)

# Create labels for each activity
reading_labels = np.full((reading_data.shape[0],), fill_value=0)  # 0 represents reading
speaking_labels = np.full((speaking_data.shape[0],), fill_value=1)  # 1 represents speaking
watching_labels = np.full((watching_data.shape[0],), fill_value=2)  # 2 represents watching

# Concatenate data and labels
X = np.concatenate((reading_data, speaking_data, watching_data), axis=0)
y = np.concatenate((reading_labels, speaking_labels, watching_labels), axis=0)

# Encode labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape data for LSTM (assuming data is 1D, modify as needed if your data has more dimensions)
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build an LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(X_train_reshaped.shape[1], 1)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output neurons for 3 classes (reading, speaking, watching)
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Make predictions on the test set
predictions = model.predict(X_test_reshaped)

# Decode labels if necessary
decoded_true_labels = label_encoder.inverse_transform(y_test)
decoded_predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

# Create a Pandas DataFrame with input data, true labels, and predicted labels
df = pd.DataFrame({'Input Data': list(X_test), 'True Label': decoded_true_labels, 'Predicted Label': decoded_predicted_labels})

# Display the DataFrame
print(df)


2023-12-21 16:21:57.751228: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-21 16:21:57.752706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 16:21:57.794314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 16:21:57.795233: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-21 16:21:58.384989: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/10


2023-12-21 16:22:01.119100: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-21 16:22:01.120294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-21 16:22:01.121135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2600/2612 [============================>.] - ETA: 0s - loss: 0.6808 - accuracy: 0.7269

2023-12-21 16:22:07.694554: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-21 16:22:07.695412: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-21 16:22:07.695997: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2612/2612 [==============================] - 7s 2ms/step - loss: 0.6800 - accuracy: 0.7273 - val_loss: 0.5704 - val_accuracy: 0.7969
Epoch 2/10
2612/2612 [==============================] - 8s 3ms/step - loss: 0.5487 - accuracy: 0.7959 - val_loss: 0.5032 - val_accuracy: 0.8132
Epoch 3/10
2612/2612 [==============================] - 8s 3ms/step - loss: 0.5019 - accuracy: 0.8187 - val_loss: 0.4727 - val_accuracy: 0.8360
Epoch 4/10
2612/2612 [==============================] - 6s 2ms/step - loss: 0.4726 - accuracy: 0.8281 - val_loss: 0.4529 - val_accuracy: 0.8398
Epoch 5/10
2612/2612 [==============================] - 6s 2ms/step - loss: 0.4354 - accuracy: 0.8359 - val_loss: 0.4000 - val_accuracy: 0.8431
Epoch 6/10
2612/2612 [==============================] - 6s 2ms/step - loss: 0.4138 - accuracy: 0.8443 - val_loss: 0.3963 - val_accuracy: 0.8484
Epoch 7/10
2612/2612 [==============================] - 6s 2ms/step - loss: 0.4008 - accuracy: 0.8510 - val_loss: 0.3807 - val_accuracy: 0.8690
Epo

2023-12-21 16:23:07.144159: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-12-21 16:23:07.144853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-12-21 16:23:07.145676: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

653/653 [==============================] - 1s 952us/step
                                              Input Data  True Label  \
0      [832.3600464, 842.2299194, -546.8758545, 835.9...           2   
1      [840.5849609, 850.454834, -722.6574097, 667.96...           2   
2      [840.5849609, 840.5849609, -222.656601, 941.40...           1   
3      [852.0997925, 838.9400024, -136.7189636, 1003....           0   
4      [850.454834, 840.5849609, -601.5634155, 816.40...           0   
...                                                  ...         ...   
20891  [842.2299194, 837.2949829, -363.2818298, 949.2...           2   
20892  [850.454834, 858.6797485, -167.9690094, 976.56...           0   
20893  [848.8098755, 899.8042603, -347.6567993, 937.5...           0   
20894  [837.2949829, 829.0700684, -152.3439941, 1000....           0   
20895  [829.0700684, 817.5552368, -562.5008545, 816.4...           0   

       Predicted Label  
0                    0  
1                    2  
2  